In [1]:
import tensorflow as tf
import keras
from tensorflow.keras import datasets, models, layers

from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

import os
import av
import shutil
import imghdr
from PIL import Image
import pickle

2023-03-28 08:21:34.785201: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 08:21:48.319517: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-28 08:21:48.319726: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-28 08:23:00.046379: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
DS_CDFV1 = 'celeb_df_v1/'
DS_CDFV2 = 'celeb_df_v2/'

DS_ORGINAL = 'dataset_original/'
DS_SPLIT = 'dataset_split/'
DS_IFRAMES = 'dataset_iframes/'
DS_FACE = 'dataset_face/'
DS_FACE_IMG = 'dataset_face_img/'
DS_SRM_SNIPPETS = 'dataset_srm_snippets_5/'
DS_SEGMENTS = 'dataset_segments/'
DS_RAW = 'dataset_raw/'
DS_RESIDUALS = 'dataset_residuals/'
DS_TEMPORAL = 'dataset_temporal/'

MODELS = 'models/'


SEG_1 = 'seg_1/'
SEG_2 = 'seg_2/'
SEG_3 = 'seg_3/'
SEG_4 = 'seg_4/'
SEG_5 = 'seg_5/'

SEG = ['seg_1_', 'seg_2_', 'seg_3_', 'seg_4_', 'seg_5_']

DS_TRAIN = 'train_dataset/'
DS_TEST = 'test_dataset/'
DS_VAL = 'val_dataset/'

CLASS_FAKE = 'fake/'
CLASS_REAL = 'real/'


TOP_LEVEL_1 = [DS_SPLIT, DS_IFRAMES, DS_FACE, DS_FACE_IMG, DS_SRM_SNIPPETS]
TOP_LEVEL_2 = [DS_SEGMENTS, DS_RAW, DS_RESIDUALS]
SEGMENTS = [SEG_1, SEG_2, SEG_3, SEG_4, SEG_5]
SPLIT = [DS_TRAIN, DS_TEST, DS_VAL]
CLASS = [CLASS_REAL, CLASS_FAKE]

DATASET = [DS_CDFV1, DS_CDFV2]

In [4]:
os.makedirs(MODELS, exist_ok=True)

# Frame Extraction

In [6]:
for dataset in DATASET:
    for split in SPLIT:
        if split is DS_TEST:
            for segment in SEGMENTS:
                for label_class in CLASS:
                    dir = dataset + DS_TEMPORAL + split + segment + label_class
                    os.makedirs(dir, exist_ok=True) 
        else:
            for label_class in CLASS:
                dir = dataset + DS_TEMPORAL + split + label_class
                os.makedirs(dir, exist_ok=True)

In [16]:
# List of testing videos
for src_dir in DATASET:
    for label, class_label in enumerate(CLASS):
        with open(src_dir + DS_TEMPORAL + 'testing_videos.txt', 'a+') as f:
            for file in os.listdir(src_dir + DS_SPLIT + DS_TEST + class_label):
                f.write(f'{file} {label}\n')

### Test Dataset

In [7]:
def extract_frames_test(ds, dest_dir, segment, split, label, file):
    src_dir = ds + DS_RESIDUALS + segment + split + label

    for index, curr_segment in enumerate(SEGMENTS):
        if curr_segment > segment:
            break

        residual_file = src_dir + SEG[index] + file
        os.makedirs(dest_dir + SEGMENTS[index], exist_ok=True)

        vid = av.open(residual_file)

        for count, frame in enumerate(vid.decode()):
            image = frame.to_image()
            image.save(f'{dest_dir}{SEGMENTS[index]}frame_{count}.jpg')

In [8]:
def extract_segment_frames(ds, segment):
    file = open(ds + DS_TEMPORAL + 'testing_videos.txt', 'r')
    videos = file.readlines()

    for video in videos:
        [file, label] = video.split(' ')
        label = CLASS_REAL if int(label) == 0 else CLASS_FAKE
        
        dest_dir = ds + DS_TEMPORAL + DS_TEST + segment + label + '/' + file +'/'
        os.makedirs(dest_dir, exist_ok=True)

        extract_frames_test(ds, dest_dir, segment, DS_TEST, label, file)    

In [6]:
extract_segment_frames(DS_CDFV1, SEG_2)

### Training and Validation Dataset

In [5]:
def extract_frames_train_val(ds):
    for split in SPLIT:
        if split == DS_TEST:
            continue
        else:
            for label_class in CLASS:
                src_dir = ds + DS_RESIDUALS + SEG_1 + split + label_class
                dest_dir = ds + DS_TEMPORAL + split + label_class
            
                for id, video in enumerate(os.listdir(src_dir)):
                    vid = av.open(src_dir + video)

                    for count, frame in enumerate(vid.decode()):
                        image = frame.to_image()
                        image.save(f'{dest_dir}frame_{count}_vid_{id}.jpg')

In [6]:
extract_frames_train_val(DS_CDFV1)

# ResNet-18 Model

In [3]:
class ResnetBlock(Model):

    def __init__(self, channels: int, down_sample = False):
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)

        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras)
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        
        self.bn_1 = BatchNormalization()

        self.conv_2 = Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        
        self.bn_2 = BatchNormalization()

        self.merge = Add()

        if self.__down_sample:
            # perform down sampling using stride of 2
            self.res_conv = Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            
            self.res_bn = BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)

        return out

In [4]:
class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        super().__init__(**kwargs)

        self.conv_1 = Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        
        self.init_bn = BatchNormalization()

        self.pool_2 = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")

        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)

        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)

        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)

        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)

        self.avg_pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)

        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)

        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        
        return out

In [5]:
input_shape = (None, 224, 224, 3)

model = ResNet18(2)
model.build(input_shape)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001,
    decay_steps = 10,
    decay_rate = 0.1,
    staircase = False)

model.compile(optimizer = Adam(learning_rate = lr_schedule), 
              loss = 'categorical_crossentropy', 
              metrics = [keras.metrics.CategoricalAccuracy(), 
                         keras.metrics.Precision(), 
                         keras.metrics.Recall(),
                         keras.metrics.AUC()])
model.summary()

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2023-03-27 20:11:47.168757: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/c/Users/Kugan raju/Desktop/CODE/FFmpeg/my_FFmpeg/lib/
2023-03-27 20:11:47.169004: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-27 20:11:47.169146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-O3QSBC4): /proc/driver/nvidia/version does not exist
2023-03-27 20:11:47.169724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compile

Model: "res_net18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  9472      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 resnet_block (ResnetBlock)  multiple                  74368     
                                                                 
 resnet_block_1 (ResnetBlock  multiple                 74368     
 )                                                               
                                                         

# Training

In [6]:
def get_train_val_dataset(train_src, val_src):

  train_ds = keras.utils.image_dataset_from_directory(
        directory = train_src,
        labels = 'inferred',
        label_mode = 'categorical',
        batch_size = 256,
        image_size = (224, 224),
        color_mode = 'rgb',
        shuffle = True,
        seed = 1
    )

  for data, labels in train_ds.take(1):
    print(data.shape)

  val_ds = keras.utils.image_dataset_from_directory(
        directory = val_src,
        labels = 'inferred',
        label_mode = 'categorical',
        batch_size = 256,
        image_size = (224, 224),
        color_mode = 'rgb',
        shuffle = True,
        seed = 1
    )

  for data, labels in val_ds.take(1):
    print(data.shape)

  return train_ds, val_ds

In [7]:
train_src = DS_CDFV1 + DS_TEMPORAL + DS_TRAIN
val_src = DS_CDFV1 + DS_TEMPORAL + DS_VAL

In [ ]:
# Load from checkpoint (if exists)
try:
    saved_model = keras.models.load_model(MODELS + 'temporal_stream')
    model = saved_model

except IOError:
    pass

In [8]:
train_ds, val_ds = get_train_val_dataset(train_src, val_src)

Found 359405 files belonging to 2 classes.
(32, 224, 224, 3)
Found 89167 files belonging to 2 classes.
(32, 224, 224, 3)


In [9]:
max_epochs = 5

In [10]:
history = model.fit(train_ds, 
          epochs = max_epochs, 
          validation_data = val_ds,
          callbacks = keras.callbacks.ModelCheckpoint(MODELS + 'temporal_stream'),
          verbose = 1)

Epoch 1/5
    3/11232 [..............................] - ETA: 13:39:26 - loss: 4.1035 - categorical_accuracy: 0.4167 - precision: 0.4167 - recall: 0.4167 - auc: 0.3837

KeyboardInterrupt: 

In [ ]:
with open(MODELS + '/history', 'wb') as f:
    pickle.dump(history, f)

In [ ]:
with open(MODELS + '/history', 'rb') as f:
    history = pickle.load(f)

# Testing